In [13]:
# תא 1: התקנת ספריות (מעודכן)

# הסרת הספרייה הישנה והתקנת החדשה
!pip uninstall pinecone-client -y
!pip install pinecone openai pandas tiktoken tqdm

print("✅ ספריות הותקנו!")

✅ ספריות הותקנו!


In [46]:
# תא 2: הגדרות ו-API Keys
import os

# ══════════════════════════════════════════════════════════
#                 🔑 הכנס את ה-API Keys שלך
# ══════════════════════════════════════════════════════════

# ⚠️ חשוב: אל תדחף מפתחות API ל-GitHub!
# השתמש במשתני סביבה או בקובץ .env (שמופיע ב-.gitignore)
OPENAI_API_KEY = os.environ.get("LLMOD_API_KEY") or "YOUR_API_KEY_HERE"       # מ-app.llmodel.ai
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY") or "YOUR_PINECONE_API_KEY_HERE"     # מ-pinecone.io

# Base URL (אם צריך - תבדוק ב-app.llmodel.ai)
OPENAI_BASE_URL = "https://api.llmod.ai"  # או None אם לא צריך

# ══════════════════════════════════════════════════════════
#                 ⚙️ מצב עבודה
# ══════════════════════════════════════════════════════════

TEST_MODE = False        # True = בדיקות, False = production
TEST_SIZE = 200          # כמות הרצאות בבדיקות

# ══════════════════════════════════════════════════════════
#                 📊 Hyperparameters
# ══════════════════════════════════════════════════════════

CHUNK_SIZE = 512        # מקסימום: 2048
OVERLAP_RATIO = 0.15    # מקסימום: 0.3
TOP_K = 8              # מקסימום: 30

# ══════════════════════════════════════════════════════════
#                 🔧 הגדרות נוספות (לא לשנות)
# ══════════════════════════════════════════════════════════
# חדש - להחליף:
EMBEDDING_MODEL = "RPRTHPB-text-embedding-3-small"
CHAT_MODEL = "RPRTHPB-gpt-5-mini"

EMBEDDING_DIMENSIONS = 1536

# שם Index לפי מצב העבודה
if TEST_MODE:
    INDEX_NAME = "ted-talks-test"
else:
    INDEX_NAME = "ted-talks-rag"

# הגדר משתני סביבה
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

print("✅ הגדרות נטענו!")
print(f"   📦 מצב: {'בדיקות' if TEST_MODE else 'Production'}")
print(f"   📊 Chunk size: {CHUNK_SIZE}")
print(f"   📊 Overlap: {OVERLAP_RATIO}")
print(f"   📊 Top-K: {TOP_K}")
print(f"   🗄️ Index name: {INDEX_NAME}")

✅ הגדרות נטענו!
   📦 מצב: Production
   📊 Chunk size: 512
   📊 Overlap: 0.15
   📊 Top-K: 8
   🗄️ Index name: ted-talks-rag


In [47]:
# תא 3: טעינת ספריות ואתחול

import pandas as pd
import tiktoken
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
import time
import json

# אתחול OpenAI client
if OPENAI_BASE_URL:
    client = OpenAI(
        api_key=OPENAI_API_KEY,
        base_url=OPENAI_BASE_URL
    )
else:
    client = OpenAI(api_key=OPENAI_API_KEY)

# אתחול Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# אתחול tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

print("✅ ספריות נטענו בהצלחה!")
print(f"   OpenAI Base URL: {OPENAI_BASE_URL or 'Default (api.openai.com)'}")

✅ ספריות נטענו בהצלחה!
   OpenAI Base URL: https://api.llmod.ai


In [48]:
# תא 4: טעינת הנתונים

# ══════════════════════════════════════════════════════════
#                 📁 נתיב לקובץ CSV
# ══════════════════════════════════════════════════════════

CSV_PATH = "/Users/patze/PycharmProjects/rag_model_project/ted_talks_en.csv"

# ══════════════════════════════════════════════════════════

df_full = pd.read_csv(CSV_PATH)

if TEST_MODE:
    df = df_full.head(TEST_SIZE)
    print(f"🧪 מצב בדיקות: {len(df)} הרצאות מתוך {len(df_full)}")
else:
    df = df_full
    print(f"🚀 מצב Production: {len(df)} הרצאות")

print(f"\n📋 עמודות: {list(df.columns)}")
print(f"\n🎤 דוגמה ראשונה:")
print(f"   כותרת: {df['title'].iloc[0]}")
print(f"   דובר: {df['speaker_1'].iloc[0]}")

🚀 מצב Production: 4005 הרצאות

📋 עמודות: ['talk_id', 'title', 'speaker_1', 'all_speakers', 'occupations', 'about_speakers', 'views', 'recorded_date', 'published_date', 'event', 'native_lang', 'available_lang', 'comments', 'duration', 'topics', 'related_talks', 'url', 'description', 'transcript']

🎤 דוגמה ראשונה:
   כותרת: Averting the climate crisis
   דובר: Al Gore


In [49]:
# תא 5: פונקציות עזר

def count_tokens(text: str) -> int:
    """ספירת טוקנים בטקסט"""
    return len(tokenizer.encode(text))


def chunk_text(text: str, chunk_size: int, overlap_ratio: float) -> list:
    """
    חלוקת טקסט ל-chunks עם חפיפה

    Args:
        text: הטקסט לחלוקה
        chunk_size: גודל כל chunk בטוקנים
        overlap_ratio: אחוז החפיפה בין chunks

    Returns:
        רשימת chunks
    """
    tokens = tokenizer.encode(text)
    overlap_size = int(chunk_size * overlap_ratio)
    step = chunk_size - overlap_size

    chunks = []
    for i in range(0, len(tokens), step):
        chunk_tokens = tokens[i:i + chunk_size]
        if len(chunk_tokens) < 50:  # דלג על chunks קטנים מדי
            continue
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)

    return chunks


def estimate_chunks_and_cost(dataframe, chunk_size, overlap_ratio):
    """הערכת כמות chunks ועלות"""
    total_chunks = 0
    total_tokens = 0

    for _, row in dataframe.iterrows():
        transcript = str(row['transcript'])
        if pd.isna(row['transcript']) or transcript == 'nan':
            continue

        tokens = tokenizer.encode(transcript)
        overlap_size = int(chunk_size * overlap_ratio)
        step = chunk_size - overlap_size

        num_chunks = max(1, (len(tokens) - chunk_size) // step + 1)
        total_chunks += num_chunks
        # כל chunk + metadata (~100 tokens)
        total_tokens += num_chunks * (chunk_size + 100)

    cost = (total_tokens / 1_000_000) * 0.02
    return total_chunks, total_tokens, cost


# בדיקה
test_text = "This is a test sentence. " * 100
test_chunks = chunk_text(test_text, CHUNK_SIZE, OVERLAP_RATIO)
print(f"✅ פונקציות עזר מוכנות!")
print(f"   טקסט בדיקה: {count_tokens(test_text)} tokens → {len(test_chunks)} chunks")

✅ פונקציות עזר מוכנות!
   טקסט בדיקה: 601 tokens → 2 chunks


In [50]:
# תא 6: השוואת Hyperparameters
# הרץ את התא הזה לפני שמחליטים על הפרמטרים הסופיים

options = [
    {"chunk_size": 256,  "overlap": 0.10},
    {"chunk_size": 256,  "overlap": 0.20},
    {"chunk_size": 512,  "overlap": 0.15},
    {"chunk_size": 512,  "overlap": 0.20},
    {"chunk_size": 768,  "overlap": 0.15},
    {"chunk_size": 1024, "overlap": 0.15},
    {"chunk_size": 1024, "overlap": 0.20},
]

print("📊 השוואת Hyperparameters (על כל הנתונים):")
print("=" * 65)
print(f"{'Chunk Size':<12} {'Overlap':<10} {'Chunks':<12} {'Tokens':<15} {'Cost':<10}")
print("-" * 65)

for opt in options:
    chunks, tokens, cost = estimate_chunks_and_cost(
        df_full, opt["chunk_size"], opt["overlap"]
    )
    marker = " ◀️" if opt["chunk_size"] == CHUNK_SIZE and opt["overlap"] == OVERLAP_RATIO else ""
    print(f"{opt['chunk_size']:<12} {opt['overlap']:<10} {chunks:<12} {tokens:<15,} ${cost:.4f}{marker}")

print("=" * 65)
print(f"\n💡 הבחירה הנוכחית שלך: chunk_size={CHUNK_SIZE}, overlap={OVERLAP_RATIO}")

# הערכה לנתונים הנוכחיים (test או full)
chunks, tokens, cost = estimate_chunks_and_cost(df, CHUNK_SIZE, OVERLAP_RATIO)
print(f"\n📦 עבור {'בדיקות' if TEST_MODE else 'Production'} ({len(df)} הרצאות):")
print(f"   Chunks: {chunks:,}")
print(f"   Tokens: {tokens:,}")
print(f"   עלות משוערת: ${cost:.4f}")

📊 השוואת Hyperparameters (על כל הנתונים):
Chunk Size   Overlap    Chunks       Tokens          Cost      
-----------------------------------------------------------------
256          0.1        35925        12,789,300      $0.2558
256          0.2        40163        14,298,028      $0.2860
512          0.15       17623        10,785,276      $0.2157 ◀️
512          0.2        18622        11,396,664      $0.2279
768          0.15       11286        9,796,248       $0.1959
1024         0.15       8255         9,278,620       $0.1856
1024         0.2        8600         9,666,400       $0.1933

💡 הבחירה הנוכחית שלך: chunk_size=512, overlap=0.15

📦 עבור Production (4005 הרצאות):
   Chunks: 17,623
   Tokens: 10,785,276
   עלות משוערת: $0.2157


In [51]:
# תא 7: יצירת Chunks

all_chunks = []

print(f"🔄 מעבד {len(df)} הרצאות...")

for idx, row in tqdm(df.iterrows(), total=len(df), desc="מעבד"):
    talk_id = str(row['talk_id'])
    title = str(row['title'])
    speaker = str(row['speaker_1'])
    topics = str(row['topics'])
    description = str(row['description'])
    transcript = str(row['transcript'])

    # דלג אם אין תמליל
    if pd.isna(row['transcript']) or transcript == 'nan' or len(transcript) < 100:
        continue

    # Metadata prefix לשיפור החיפוש
    metadata_prefix = f"Title: {title}\nSpeaker: {speaker}\nTopics: {topics}\n\n"

    # חלק את התמליל
    chunks = chunk_text(transcript, CHUNK_SIZE, OVERLAP_RATIO)

    for chunk_idx, chunk in enumerate(chunks):
        text_for_embedding = metadata_prefix + chunk

        all_chunks.append({
            "id": f"{talk_id}_{chunk_idx}",
            "text_for_embedding": text_for_embedding,
            "metadata": {
                "talk_id": talk_id,
                "title": title,
                "speaker": speaker,
                "topics": topics,
                "description": description[:500],  # קיצור ל-500 תווים
                "chunk_index": chunk_idx,
                "chunk_text": chunk
            }
        })

# סיכום
total_tokens = sum(count_tokens(c["text_for_embedding"]) for c in all_chunks)
estimated_cost = (total_tokens / 1_000_000) * 0.02

print(f"\n✅ נוצרו {len(all_chunks):,} chunks")
print(f"💰 עלות embedding משוערת: ${estimated_cost:.4f}")
print(f"   סה״כ טוקנים: {total_tokens:,}")

🔄 מעבד 4005 הרצאות...


מעבד: 100%|██████████| 4005/4005 [00:01<00:00, 2254.10it/s]



✅ נוצרו 21,761 chunks
💰 עלות embedding משוערת: $0.2249
   סה״כ טוקנים: 11,243,924


In [52]:
# תא 8: יצירת Index ב-Pinecone

existing_indexes = [idx.name for idx in pc.list_indexes()]
print(f"📋 Indexes קיימים: {existing_indexes}")

# בדיקה אם ה-Index כבר קיים
if INDEX_NAME in existing_indexes:
    print(f"\n⚠️ Index '{INDEX_NAME}' כבר קיים!")
    user_input = input("   למחוק וליצור מחדש? (yes/no): ").strip().lower()

    if user_input == "yes":
        print(f"🗑️ מוחק '{INDEX_NAME}'...")
        pc.delete_index(INDEX_NAME)
        time.sleep(5)
    else:
        print("✅ משתמש ב-Index הקיים")
        index = pc.Index(INDEX_NAME)
        stats = index.describe_index_stats()
        print(f"   Vectors קיימים: {stats.total_vector_count}")

# יצירת Index חדש אם צריך
if INDEX_NAME not in [idx.name for idx in pc.list_indexes()]:
    print(f"\n📦 יוצר Index '{INDEX_NAME}'...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIMENSIONS,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

    print("⏳ ממתין שה-Index יהיה מוכן (30 שניות)...")
    time.sleep(30)
    print(f"✅ Index '{INDEX_NAME}' נוצר!")

# התחברות ל-Index
index = pc.Index(INDEX_NAME)
print(f"\n✅ מחובר ל-Index: {INDEX_NAME}")

📋 Indexes קיימים: ['ted-talks-test']

📦 יוצר Index 'ted-talks-rag'...
⏳ ממתין שה-Index יהיה מוכן (30 שניות)...
✅ Index 'ted-talks-rag' נוצר!

✅ מחובר ל-Index: ted-talks-rag


In [53]:
# תא 9: יצירת Embeddings והעלאה
# ⚠️ תא זה עולה כסף! וודא שאתה מוכן

BATCH_SIZE = 100

print("=" * 60)
print("🚀 התחלת תהליך Embedding והעלאה")
print("=" * 60)
print(f"   Chunks: {len(all_chunks):,}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Index: {INDEX_NAME}")
print("=" * 60)

# אישור לפני התחלה
confirm = input("\n🤔 להתחיל? (yes/no): ").strip().lower()
if confirm != "yes":
    print("❌ בוטל על ידי המשתמש")
else:
    total_batches = (len(all_chunks) + BATCH_SIZE - 1) // BATCH_SIZE
    errors = []

    for i in tqdm(range(0, len(all_chunks), BATCH_SIZE), total=total_batches, desc="📤 מעלה"):
        batch = all_chunks[i:i + BATCH_SIZE]
        texts = [chunk["text_for_embedding"] for chunk in batch]

        try:
            # יצירת embeddings
            response = client.embeddings.create(
                model=EMBEDDING_MODEL,
                input=texts,
                dimensions=EMBEDDING_DIMENSIONS
            )

            # הכנת vectors
            vectors = []
            for j, embedding_data in enumerate(response.data):
                chunk = batch[j]
                vectors.append({
                    "id": chunk["id"],
                    "values": embedding_data.embedding,
                    "metadata": chunk["metadata"]
                })

            # העלאה ל-Pinecone
            index.upsert(vectors=vectors)

        except Exception as e:
            errors.append({"batch": i // BATCH_SIZE, "error": str(e)})
            print(f"\n❌ שגיאה ב-batch {i//BATCH_SIZE}: {e}")
            continue

        time.sleep(0.1)  # מניעת rate limiting

    print(f"\n✅ ההעלאה הושלמה!")
    if errors:
        print(f"⚠️ היו {len(errors)} שגיאות")

    # סטטיסטיקות
    stats = index.describe_index_stats()
    print(f"📊 סה״כ vectors ב-Index: {stats.total_vector_count:,}")

🚀 התחלת תהליך Embedding והעלאה
   Chunks: 21,761
   Batch size: 100
   Index: ted-talks-rag


📤 מעלה: 100%|██████████| 218/218 [13:51<00:00,  3.81s/it]


✅ ההעלאה הושלמה!
📊 סה״כ vectors ב-Index: 21,761


In [54]:
# תא 10: בדיקת חיפוש

def search_ted_talks(query: str, top_k: int = TOP_K):
    """חיפוש הרצאות TED"""
    # יצירת embedding לשאילתה
    response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
        dimensions=EMBEDDING_DIMENSIONS
    )
    query_embedding = response.data[0].embedding

    # חיפוש ב-Pinecone
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    return results.matches

# בדיקות
test_queries = [
    "climate change and environment",
    "technology improving lives",
    "education and learning",
    "overcoming fear and anxiety",
]

print("🔍 בדיקת חיפוש:")
print("=" * 60)

for query in test_queries:
    print(f"\n📝 שאילתה: '{query}'")
    results = search_ted_talks(query, top_k=3)

    for i, match in enumerate(results, 1):
        title = match.metadata.get('title', 'Unknown')
        speaker = match.metadata.get('speaker', 'Unknown')
        score = match.score
        print(f"   {i}. {title} ({speaker}) - Score: {score:.4f}")

print("\n" + "=" * 60)
print("✅ החיפוש עובד!")

🔍 בדיקת חיפוש:

📝 שאילתה: 'climate change and environment'
   1. Climate change is happening. Here's how we adapt (Alice Bows-Larkin) - Score: 0.5313
   2. Climate change is happening. Here's how we adapt (Alice Bows-Larkin) - Score: 0.5288
   3. The case for optimism on climate change (Al Gore) - Score: 0.4907

📝 שאילתה: 'technology improving lives'
   1. Medicine's future? There's an app for that (Daniel Kraft) - Score: 0.5179
   2. Medicine's future? There's an app for that (Daniel Kraft) - Score: 0.5169
   3. Welcome to the age of the industrial internet (Marco Annunziata) - Score: 0.5098

📝 שאילתה: 'education and learning'
   1. What we're learning from online education (Daphne Koller) - Score: 0.4215
   2. What we're learning from online education (Daphne Koller) - Score: 0.4197
   3. The global learning crisis -- and what to do about it (Amel Karboul) - Score: 0.4174

📝 שאילתה: 'overcoming fear and anxiety'
   1. How to cope with anxiety (Olivia Remes) - Score: 0.5894
   2. How 

In [55]:
# תא 11: בדיקת RAG מלא

# System Prompt כנדרש במטלה
SYSTEM_PROMPT = """You are a TED Talk assistant that answers questions strictly and only based on the TED dataset context provided to you (metadata and transcript passages). You must not use any external knowledge, the open internet, or information that is not explicitly contained in the retrieved context. If the answer cannot be determined from the provided context, respond: "I don't know based on the provided TED data." Always explain your answer using the given context, quoting or paraphrasing the relevant transcript or metadata when helpful."""


def ask_ted_rag(question: str):
    """שאילתת RAG מלאה"""

    # 1. חיפוש chunks רלוונטיים
    results = search_ted_talks(question, top_k=TOP_K)

    # 2. בניית context
    context_parts = []
    for i, match in enumerate(results, 1):
        context_parts.append(f"""[Context {i}]
Talk ID: {match.metadata.get('talk_id')}
Title: "{match.metadata.get('title')}"
Speaker: {match.metadata.get('speaker')}
Topics: {match.metadata.get('topics')}
Transcript excerpt:
{match.metadata.get('chunk_text', '')}
""")

    context_string = "\n---\n".join(context_parts)

    # 3. בניית User Prompt
    user_prompt = f"""Based on the following TED Talk context, please answer the question.

CONTEXT:
{context_string}

QUESTION: {question}

Please provide a helpful answer based only on the context above."""

    # 4. קריאה ל-LLM
    chat_response = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1000
    )

    answer = chat_response.choices[0].message.content

    # 5. החזרת תוצאה מלאה
    return {
        "response": answer,
        "context": [
            {
                "talk_id": m.metadata.get('talk_id'),
                "title": m.metadata.get('title'),
                "chunk": m.metadata.get('chunk_text', '')[:200] + "...",
                "score": m.score
            }
            for m in results
        ],
        "Augmented_prompt": {
            "System": SYSTEM_PROMPT,
            "User": user_prompt
        }
    }


# בדיקה
print("🎤 בדיקת RAG מלא")
print("=" * 60)

test_question = "Find a TED talk that discusses climate change. Provide the title and speaker."
print(f"\n❓ שאלה: {test_question}\n")

result = ask_ted_rag(test_question)

print("💬 תשובה:")
print(result["response"])

print("\n📚 Context שנשלף:")
for ctx in result["context"]:
    print(f"   • {ctx['title']} (Score: {ctx['score']:.4f})")

print("\n✅ RAG עובד!")

🎤 בדיקת RAG מלא

❓ שאלה: Find a TED talk that discusses climate change. Provide the title and speaker.

💬 תשובה:
Title: "The most important thing you can do to fight climate change: talk about it"
Speaker: Katharine Hayhoe

I know this from the provided TED transcript, where Hayhoe opens and closes by urging conversation about climate change — e.g., "The bottom line is this: climate change is affecting you and me right here, right now, in the places where we live" — and the talk's topics include "climate change" and "communication."

📚 Context שנשלף:
   • The most important thing you can do to fight climate change: talk about it (Score: 0.6072)
   • How the military fights climate change (Score: 0.5757)
   • How we can turn the tide on climate (Score: 0.5692)
   • New thinking on the climate crisis (Score: 0.5677)
   • The most important thing you can do to fight climate change: talk about it (Score: 0.5645)
   • The most important thing you can do to fight climate change: talk about i

In [56]:
# תא 12: בדיקות לפי דרישות המטלה

print("🧪 בדיקות לפי 4 הקטגוריות מהמטלה")
print("=" * 70)

# 1. Precise Fact Retrieval
print("\n1️⃣ Precise Fact Retrieval")
print("-" * 50)
q1 = "Find a TED talk that discusses overcoming fear or anxiety. Provide the title and speaker."
r1 = ask_ted_rag(q1)
print(f"❓ {q1}")
print(f"💬 {r1['response'][:500]}...")

# 2. Multi-Result Topic Listing
print("\n2️⃣ Multi-Result Topic Listing")
print("-" * 50)
q2 = "Which TED talks focus on education or learning? Return a list of exactly 3 talk titles."
r2 = ask_ted_rag(q2)
print(f"❓ {q2}")
print(f"💬 {r2['response'][:500]}...")

# 3. Key Idea Summary
print("\n3️⃣ Key Idea Summary Extraction")
print("-" * 50)
q3 = "Find a TED talk where the speaker talks about technology improving people's lives. Provide the title and a short summary of the key idea."
r3 = ask_ted_rag(q3)
print(f"❓ {q3}")
print(f"💬 {r3['response'][:500]}...")

# 4. Recommendation with Justification
print("\n4️⃣ Recommendation with Evidence-Based Justification")
print("-" * 50)
q4 = "I'm looking for a TED talk about climate change and what individuals can do in their daily lives. Which talk would you recommend?"
r4 = ask_ted_rag(q4)
print(f"❓ {q4}")
print(f"💬 {r4['response'][:500]}...")

print("\n" + "=" * 70)
print("✅ כל הבדיקות הושלמו!")

🧪 בדיקות לפי 4 הקטגוריות מהמטלה

1️⃣ Precise Fact Retrieval
--------------------------------------------------
❓ Find a TED talk that discusses overcoming fear or anxiety. Provide the title and speaker.
💬 "The science of stage fright (and how to overcome it)" — Mikael Cho.

This talk explicitly addresses overcoming performance fear: the speaker asks "How do we fight it?" and recommends perspective, practice ("Practice a lot, starting long before in an environment similar to the real performance"), and last-minute tricks to "fight dirty and trick your brain" such as stretching your arms up and breathing deeply to trigger a relaxation response (Transcript excerpts in Context 1 and 2)....

2️⃣ Multi-Result Topic Listing
--------------------------------------------------
❓ Which TED talks focus on education or learning? Return a list of exactly 3 talk titles.
💬 - How to escape education's death valley — Sir Ken Robinson discusses problems in schooling and says "education under 'No Child Le

In [57]:
# תא 13: שמירת הגדרות לקובץ (לשימוש ב-Vercel)

config = {
    "chunk_size": CHUNK_SIZE,
    "overlap_ratio": OVERLAP_RATIO,
    "top_k": TOP_K,
    "index_name": INDEX_NAME,
    "embedding_model": EMBEDDING_MODEL,
    "chat_model": CHAT_MODEL,
    "total_chunks": len(all_chunks),
}

# שמירה לקובץ
config_path = "/Users/patze/PycharmProjects/rag_model_project/rag_config.json"
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)

print("✅ הגדרות נשמרו!")
print(f"📁 קובץ: {config_path}")
print(f"\n📋 תוכן:")
print(json.dumps(config, indent=2))

print("\n" + "=" * 60)
print("🎉 סיימת את חלק ה-Notebook!")
print("=" * 60)
print("""
השלבים הבאים:
1. ודא שכל הבדיקות עובדות
2. אם TEST_MODE=True, שנה ל-False והרץ שוב תאים 4-9
3. המשך לפרויקט Next.js ו-Deploy ל-Vercel
""")

✅ הגדרות נשמרו!
📁 קובץ: /Users/patze/PycharmProjects/rag_model_project/rag_config.json

📋 תוכן:
{
  "chunk_size": 512,
  "overlap_ratio": 0.15,
  "top_k": 8,
  "index_name": "ted-talks-rag",
  "embedding_model": "RPRTHPB-text-embedding-3-small",
  "chat_model": "RPRTHPB-gpt-5-mini",
  "total_chunks": 21761
}

🎉 סיימת את חלק ה-Notebook!

השלבים הבאים:
1. ודא שכל הבדיקות עובדות
2. אם TEST_MODE=True, שנה ל-False והרץ שוב תאים 4-9
3. המשך לפרויקט Next.js ו-Deploy ל-Vercel

